In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import numpy as np
from datetime import datetime
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from scipy import stats
import math
from PIL import Image

Functions

In [ ]:
#Function to convert categorical data 
def convert_categorical(column_names):
    for column_name in column_names:
        df[column_name] = pd.Categorical(df[column_name])
        df[column_name] = df[column_name].cat.codes